<h1>Assignment 3</h1>

Florian Klement, David Siegl

In [142]:
import igraph
import pandas as pd
import seaborn as sns
import tweepy
import oauth2 as oauth
import nltk
import re
import string
import numpy as np
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
api_key = "X"
api_secret = "X"
access_token_key = "X-X"
access_token_secret = "X"

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [17]:
tweets_list = tweepy.Cursor(api.search_tweets, q="#musk", tweet_mode='extended').items(1000)

In [18]:
output = []
for tweet in tweets_list:
    text = tweet._json["full_text"]
    favourite_count = tweet.favorite_count
    retweet_count = tweet.retweet_count
    created_at = tweet.created_at
    language = tweet.lang
    is_retweet = tweet.retweeted
    user_name = tweet.user.screen_name
    
    
    line = {"text" : text, "favourite_count" : favourite_count, "retweet_count" : retweet_count, "created_at" : created_at, "language" : language, "is_retweet": is_retweet, "user_name" : user_name}
    output.append(line)
    
df_tweets = pd.DataFrame(output)

In [19]:
df_tweets.to_csv("tweets_musk.csv", header = ["text", "favourite_count", "retweet_count", "created_at", "language", "is_retweet", "user_name"])

In [183]:
df_tweets_elonmusk = pd.read_csv("tweets_elonmusk.csv", index_col = 0)
df_tweets_elonmusktwitter = pd.read_csv("tweets_elonmusktwitter.csv", index_col = 0)
df_tweets_musk = pd.read_csv("tweets_musk.csv", index_col = 0)
df_tweets_tesla = pd.read_csv("tweets_tesla.csv", index_col = 0)

In [184]:
df_tweets_elonmusk['hashtag'] = '#elonmusk'
df_tweets_elonmusktwitter['hashtag'] = '#elonmusktwitter'
df_tweets_musk['hashtag'] = '#musk'
df_tweets_tesla['hashtag'] = '#tesla'

In [185]:
df_tweets_all = pd.concat([df_tweets_elonmusk, df_tweets_elonmusktwitter, df_tweets_musk, df_tweets_tesla], ignore_index=False)

In [225]:
en_tweets = df_tweets_all.loc[(df_tweets_all["language"] == "en")]
en_tweets['rt_mentions'] = en_tweets['text'].str.extract(r'(RT @\w*:)')
en_tweets['mentions'] = en_tweets['text'].str.findall(r'(@\w*)+')
en_tweets['hashtags'] = en_tweets['text'].str.findall(r'(#\w*)+')
en_tweets['rt_mentions'] = en_tweets['rt_mentions'].str.replace(r'(RT @)', '')
en_tweets['rt_mentions'] = en_tweets['rt_mentions'].str.replace(r'(:)', '')

/tmp/ipykernel_4624/2970021013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_tweets['rt_mentions'] = en_tweets['text'].str.extract(r'(RT @\w*:)')
/tmp/ipykernel_4624/2970021013.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_tweets['mentions'] = en_tweets['text'].str.findall(r'(@\w*)+')
/tmp/ipykernel_4624/2970021013.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [226]:
ls = []
for i in range(1,16):
    ls.append('mentions_' + str(i))

In [227]:
en_tweets.index = range(len(en_tweets.index))

df_test = pd.DataFrame(en_tweets['mentions'].tolist())
df_test.columns = ls
en_tweets = en_tweets.merge(df_test, how='outer', left_index=True, right_index=True)

In [231]:
ls = []
for i in range(1,28):
    ls.append('hashtags_' + str(i))

In [232]:
df_test = pd.DataFrame(en_tweets['hashtags'].tolist())
df_test.columns = ls
en_tweets = en_tweets.merge(df_test, how='outer', left_index=True, right_index=True)

In [213]:
for i in range(0,15):
    en_tweets[i] = en_tweets[i].str.replace(r'(@)', '')

/tmp/ipykernel_4624/3573310007.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  en_tweets[i] = en_tweets[i].str.replace(r'(@)', '')


In [233]:
en_tweets.to_csv("tweets_processed.csv")

In [235]:
df_networks = pd.read_csv("tweets_processed.csv", index_col = 0)

In [60]:
en_tweets = en_tweets[~en_tweets["text"].str.startswith("RT", na = False)]
en_tweets_rt = en_tweets[en_tweets["text"].str.startswith("RT", na = False)]